In [ ]:
import pandas as pd
import pymysql

# List of CSV file paths
csv_files = [r"data/ap_bus_details.csv", r"data/kerala_bus_details.csv", r"data/Telangana_bus_details.csv"]   

# Step 1: Read and concatenate all CSV files into a single DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Step 2: Add an ID column to the combined DataFrame
combined_df.insert(0, 'id', range(1, len(combined_df) + 1))

# Step 3: Clean the data
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '')  # Remove 'INR ' from Price
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract('(\d+)')  # Extract digits

# Step 4: Drop any rows with NaN values
combined_df = combined_df.dropna()

# Step 5: Save the combined DataFrame to a CSV file if needed
combined_df.to_csv("bus_routes.csv", index=False)

# Step 6: Connect to MySQL and create the table
myconnection = pymysql.connect(host='localhost', user='root', passwd='1234', database="redbus")
cursor = myconnection.cursor()

# Define table creation query based on column types
column_definitions = ", ".join(f"{col} {dtype}" for col, dtype in zip(combined_df.columns, combined_df.dtypes))
column_definitions = column_definitions.replace("float64", "FLOAT").replace("object", "TEXT").replace("int64", "INT")
table_name = "bus_routes"
cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({column_definitions});")

# Step 7: Insert data into MySQL table
for _, row in combined_df.iterrows():
    cursor.execute(f"INSERT INTO {table_name} VALUES {tuple(row)}")

# Step 8: Commit and close the connection
myconnection.commit()
cursor.close()
myconnection.close()

print("Data inserted successfully!")


Data inserted successfully!
